# Goup 2 : SF 311 Request Analysis
by Paul Pineda,  James Ye,  Tanvir Khan, Travis Le

## Project Description:

The 311 San Francsico platform serves as a tool to accept, process, and address various requests/complains from the community, such as requests for city street cleaning, graffiti, etc. In this project we use the 311-call data from the City of San Francisco to determine what are the major concerns of residents of San Francsico, what zip codes have the most 311 requests, what source was used the most to submit request, if the month or day of the week affect the number of request. and determine if there is any relation between the number of 311 calls per population, and income bracket.

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from pprint import pprint


In [2]:
# load data downloaded from SF311 database
cases_311_df = pd.read_csv("311_Cases.csv", low_memory=False)
cases_311_df

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,...,DELETE - HSOC Zones,Fix It Zones as of 2018-02-07,"CBD, BID and GBD Boundaries as of 2017",Central Market/Tenderloin Boundary,"Areas of Vulnerability, 2016",Central Market/Tenderloin Boundary Polygon - Updated,HSOC Zones as of 2018-06-05,OWED Public Spaces,Parks Alliance CPSI (27+TL sites),Neighborhoods
0,929675,07/29/2011 09:16:17 AM,11/16/2009 09:25:00 AM,11/16/2009 09:25:00 AM,Closed,Case Resolved - Closed,DPW BSM Queue,Litter Receptacles,Cans_Left_Out_24x7,Cans_Left_Out_24x7,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,8.0
1,572452,12/31/2009 05:42:50 PM,12/31/2009 07:16:33 PM,12/31/2009 07:16:33 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Overflowing_City_Receptacle_or_Dumpster,Overflowing_City_Receptacle_or_Dumpster,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,91.0
2,572435,12/31/2009 04:56:54 PM,12/31/2009 06:16:31 PM,12/31/2009 06:16:31 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Electronics,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,8.0
3,572432,12/31/2009 04:49:38 PM,12/31/2009 06:16:30 PM,12/31/2009 06:16:30 PM,Closed,See Notes tab for more details,DPW Ops Queue,Sewer Issues,Sewage_Back_Up,Outofsewervent4inch,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,90.0
4,572429,12/31/2009 04:45:22 PM,12/31/2009 07:16:32 PM,12/31/2009 07:16:32 PM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139964,11426865,09/17/2019 09:34:23 AM,09/17/2019 12:38:39 PM,09/17/2019 12:38:39 PM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,75.0
4139965,11420535,09/16/2019 03:31:00 AM,09/16/2019 03:49:42 AM,09/16/2019 03:49:42 AM,Closed,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,31.0
4139966,11426701,09/17/2019 09:11:00 AM,09/19/2019 07:14:00 AM,09/19/2019 07:14:00 AM,Closed,Case Resolved - 9-18-19,DPT Paint Shop Transit Queue,Graffiti,Graffiti on Other_enter_additional_details_below,Other_enter_additional_details_below - Offensive,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,65.0
4139967,11423878,09/16/2019 02:53:00 PM,09/16/2019 07:10:29 PM,09/16/2019 07:10:29 PM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Furniture,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,107.0


In [3]:
cases_311_df.count()

CaseID                                                  4139969
Opened                                                  4139969
Closed                                                  4060947
Updated                                                 4139969
Status                                                  4139969
Status Notes                                            4112927
Responsible Agency                                      4139969
Category                                                4139969
Request Type                                            4139969
Request Details                                         4076250
Address                                                 4139954
Street                                                  3791606
Supervisor District                                     3791626
Neighborhood                                            3787636
Police District                                         3786328
Latitude                                

In [4]:
# pick the columns we need
cases_311_slim_df = cases_311_df[["CaseID", "Opened", "Category", "Request Type","Neighborhood", 
                                  "Latitude", "Longitude", "Source", "Media URL"]]

cases_311_slim_df.head()


,CaseID,Opened,Category,Request Type,Neighborhood,Latitude,Longitude,Source,Media URL
0,929675,07/29/2011 09:16:17 AM,Litter Receptacles,Cans_Left_Out_24x7,Outer Richmond,37.780376,-122.487923,Phone,NaN
1,572452,12/31/2009 05:42:50 PM,Street and Sidewalk Cleaning,Overflowing_City_Receptacle_or_Dumpster,Portola,37.727634,-122.403595,Phone,NaN
2,572435,12/31/2009 04:56:54 PM,Street and Sidewalk Cleaning,Bulky Items,Outer Richmond,37.781925,-122.487022,Phone,NaN
3,572432,12/31/2009 04:49:38 PM,Sewer Issues,Sewage_Back_Up,Excelsior,37.719467,-122.428566,Phone,NaN
4,572429,12/31/2009 04:45:22 PM,Street and Sidewalk Cleaning,General Cleaning,Portola,37.731480,-122.410759,Phone,NaN


In [5]:
# create a new column with date in mm/dd/yyyy format
cases_311_slim_df['Opened Date'] = cases_311_slim_df["Opened"].str.slice(stop=10)
cases_311_slim_df

d:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CaseID,Opened,Category,Request Type,Neighborhood,Latitude,Longitude,Source,Media URL,Opened Date
0,929675,07/29/2011 09:16:17 AM,Litter Receptacles,Cans_Left_Out_24x7,Outer Richmond,37.780376,-122.487923,Phone,NaN,07/29/2011
1,572452,12/31/2009 05:42:50 PM,Street and Sidewalk Cleaning,Overflowing_City_Receptacle_or_Dumpster,Portola,37.727634,-122.403595,Phone,NaN,12/31/2009
2,572435,12/31/2009 04:56:54 PM,Street and Sidewalk Cleaning,Bulky Items,Outer Richmond,37.781925,-122.487022,Phone,NaN,12/31/2009
3,572432,12/31/2009 04:49:38 PM,Sewer Issues,Sewage_Back_Up,Excelsior,37.719467,-122.428566,Phone,NaN,12/31/2009
4,572429,12/31/2009 04:45:22 PM,Street and Sidewalk Cleaning,General Cleaning,Portola,37.731480,-122.410759,Phone,NaN,12/31/2009
...,...,...,...,...,...,...,...,...,...,...
4139964,11426865,09/17/2019 09:34:23 AM,Street and Sidewalk Cleaning,Bulky Items,Visitacion Valley,37.712605,-122.407304,Mobile/Open311,http://mobile311.sfgov.org/reports/11426865/ph...,09/17/2019
4139965,11420535,09/16/2019 03:31:00 AM,Street and Sidewalk Cleaning,General Cleaning,South Beach,37.784486,-122.387571,Mobile/Open311,http://mobile311.sfgov.org/reports/11420535/ph...,09/16/2019
4139966,11426701,09/17/2019 09:11:00 AM,Graffiti,Graffiti on Other_enter_additional_details_below,Merced Heights,37.714264,-122.467081,Mobile/Open311,http://mobile311.sfgov.org/reports/11426701/ph...,09/17/2019
4139967,11423878,09/16/2019 02:53:00 PM,Street and Sidewalk Cleaning,Bulky Items,Russian Hill,37.798416,-122.411209,Phone,NaN,09/16/2019


In [6]:
# create a new column with YYYY format
cases_311_slim_df['Opened Year'] = cases_311_slim_df['Opened Date'].str.slice(start =6)
cases_311_slim_df.head()

d:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CaseID,Opened,Category,Request Type,Neighborhood,Latitude,Longitude,Source,Media URL,Opened Date,Opened Year
0,929675,07/29/2011 09:16:17 AM,Litter Receptacles,Cans_Left_Out_24x7,Outer Richmond,37.780376,-122.487923,Phone,NaN,07/29/2011,2011
1,572452,12/31/2009 05:42:50 PM,Street and Sidewalk Cleaning,Overflowing_City_Receptacle_or_Dumpster,Portola,37.727634,-122.403595,Phone,NaN,12/31/2009,2009
2,572435,12/31/2009 04:56:54 PM,Street and Sidewalk Cleaning,Bulky Items,Outer Richmond,37.781925,-122.487022,Phone,NaN,12/31/2009,2009
3,572432,12/31/2009 04:49:38 PM,Sewer Issues,Sewage_Back_Up,Excelsior,37.719467,-122.428566,Phone,NaN,12/31/2009,2009
4,572429,12/31/2009 04:45:22 PM,Street and Sidewalk Cleaning,General Cleaning,Portola,37.731480,-122.410759,Phone,NaN,12/31/2009,2009


In [7]:
# we only take data from 2017 and onward
cases_2017_df = cases_311_slim_df[cases_311_slim_df["Opened Year"].astype(int) > 2016]
cases_2017_df

,CaseID,Opened,Category,Request Type,Neighborhood,Latitude,Longitude,Source,Media URL,Opened Date,Opened Year
5,10395976,01/23/2019 01:34:53 PM,Sign Repair,Sign - Missing,Mission,37.764091,-122.413002,Web,NaN,01/23/2019,2019
11,10328235,01/07/2019 07:18:11 PM,Tree Maintenance,Trees - Tree_Other,Outer Sunset,37.741627,-122.504688,Mobile/Open311,http://mobile311.sfgov.org/reports/10328235/ph...,01/07/2019,2019
12,10314300,01/04/2019 09:22:00 AM,Street Defects,Pavement_Defect,Presidio Terrace,37.785984,-122.464783,Phone,NaN,01/04/2019,2019
13,10342330,01/10/2019 05:14:10 PM,Sidewalk or Curb,Missing_Side_Sewer_Vent_Cover,Portola,37.716455,-122.400854,Mobile/Open311,http://mobile311.sfgov.org/reports/10342330/ph...,01/10/2019,2019
16,10560591,03/04/2019 12:24:00 PM,Sidewalk or Curb,Curb_or_Curb_Ramp_Defect,Parkside,37.746143,-122.487190,Phone,NaN,03/04/2019,2019
...,...,...,...,...,...,...,...,...,...,...,...
4139964,11426865,09/17/2019 09:34:23 AM,Street and Sidewalk Cleaning,Bulky Items,Visitacion Valley,37.712605,-122.407304,Mobile/Open311,http://mobile311.sfgov.org/reports/11426865/ph...,09/17/2019,2019
4139965,11420535,09/16/2019 03:31:00 AM,Street and Sidewalk Cleaning,General Cleaning,South Beach,37.784486,-122.387571,Mobile/Open311,http://mobile311.sfgov.org/reports/11420535/ph...,09/16/2019,2019
4139966,11426701,09/17/2019 09:11:00 AM,Graffiti,Graffiti on Other_enter_additional_details_below,Merced Heights,37.714264,-122.467081,Mobile/Open311,http://mobile311.sfgov.org/reports/11426701/ph...,09/17/2019,2019
4139967,11423878,09/16/2019 02:53:00 PM,Street and Sidewalk Cleaning,Bulky Items,Russian Hill,37.798416,-122.411209,Phone,NaN,09/16/2019,2019


In [8]:
# getting ready for US Census api
from config import census_key
c = Census(census_key, year=2017)
c = Census(census_key)

In [9]:
# get US Census data by Zipcode
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

In [10]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.tail(100)

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
33020,99701,17608.0,32.2,58275.0,30330.0,2481.0,14.090186
33021,99702,3025.0,23.8,57464.0,26631.0,115.0,3.801653
33022,99703,8415.0,22.9,54526.0,22746.0,448.0,5.323827
33023,99704,30.0,20.7,-666666666.0,25570.0,0.0,0.000000
33024,99705,22804.0,32.0,81045.0,34618.0,1430.0,6.270830
...,...,...,...,...,...,...,...
33115,99923,0.0,-666666666.0,-666666666.0,-666666666.0,0.0,NaN
33116,99925,901.0,45.1,48646.0,24783.0,210.0,23.307436
33117,99926,1684.0,32.8,57969.0,22927.0,224.0,13.301663
33118,99927,59.0,49.8,17981.0,11959.0,49.0,83.050847


In [11]:
# load our excel file with San Francisco Neighborhood and Zipcode
neighborhoods_311_df = pd.read_excel("Neighborhoods_from_311data.xlsx")
neighborhoods_311_df

,Neighborhood,Zipcode
0,Sherwood Forest,94127
1,Tenderloin,94102
2,Civic Center,94102
3,Lower Haight,94102
4,Downtown / Union Square,94102
...,...,...
112,McLaren Park,94134
113,Parnassus Heights,94143
114,Central Waterfront,94107
115,India Basin,94124


In [12]:
# create summary_df which contains Census data with San Francisco zipcodes only
sf_df = pd.DataFrame()
sf_df["Zipcode"] = neighborhoods_311_df['Zipcode'].unique()
sf_df["Zipcode"] = sf_df["Zipcode"].astype(str)
summary_df = pd.merge(census_pd, sf_df, on = ['Zipcode'])
summary_df

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,94102,30140.0,41.5,33552.0,43488.0,6957.0,23.082283
1,94103,26990.0,38.8,49052.0,55807.0,5870.0,21.748796
2,94104,436.0,51.3,48429.0,64267.0,84.0,19.266055
3,94105,7675.0,37.6,199364.0,154723.0,733.0,9.550489
4,94107,29920.0,35.9,143467.0,87735.0,3391.0,11.333556
5,94108,14856.0,42.8,55341.0,51359.0,3178.0,21.392030
6,94109,56587.0,36.9,79979.0,75460.0,6718.0,11.871985
7,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
8,94111,3356.0,46.1,99464.0,88610.0,661.0,19.696067
9,94112,85373.0,41.4,82692.0,31905.0,7750.0,9.077811


In [14]:
# add Zipcode to 311 Request data
cases_2017_df.head()
cases_2017_zipcode = pd.merge(cases_2017_df,neighborhoods_311_df, on="Neighborhood" )
cases_2017_zipcode


,CaseID,Opened,Category,Request Type,Neighborhood,Latitude,Longitude,Source,Media URL,Opened Date,Opened Year,Zipcode
0,10395976,01/23/2019 01:34:53 PM,Sign Repair,Sign - Missing,Mission,37.764091,-122.413002,Web,NaN,01/23/2019,2019,94110
1,11886797,01/01/2020 09:06:00 PM,General Request - PUBLIC WORKS,request_for_service,Mission,37.764840,-122.415038,Web,NaN,01/01/2020,2020,94110
2,11089240,07/05/2019 03:12:37 PM,Graffiti,Graffiti on Building_other,Mission,37.755848,-122.414551,Integrated Agency,NaN,07/05/2019,2019,94110
3,11239390,08/07/2019 01:25:51 PM,Graffiti,Graffiti on Building_other,Mission,37.755703,-122.416397,Integrated Agency,NaN,08/07/2019,2019,94110
4,11886078,01/01/2020 03:27:00 PM,Street and Sidewalk Cleaning,General Cleaning,Mission,37.769532,-122.415847,Mobile/Open311,http://mobile311.sfgov.org/reports/11886078/ph...,01/01/2020,2020,94110
...,...,...,...,...,...,...,...,...,...,...,...,...
1877877,10492776,02/15/2019 09:42:20 AM,Tree Maintenance,Trees - Tree_Other,Yerba Buena Island,37.809266,-122.366807,Web,http://mobile311.sfgov.org/reports/10492776/ph...,02/15/2019,2019,94130
1877878,11040396,06/24/2019 03:56:00 PM,General Request - TIDA,request_for_service,Yerba Buena Island,37.813225,-122.371033,Phone,NaN,06/24/2019,2019,94130
1877879,11015442,06/18/2019 08:45:00 PM,Streetlights,Streetlight - Light_Burnt_Out,Yerba Buena Island,37.809368,-122.370087,Phone,NaN,06/18/2019,2019,94130
1877880,10829747,05/07/2019 01:04:00 PM,Street Defects,Pavement_Defect,Yerba Buena Island,37.813225,-122.371033,Phone,NaN,05/07/2019,2019,94130


In [15]:
# merge 311 data(cases_2017_zipcode) with Census data (summary_df)
# save this final_df to a csv file for later use
summary_df['Zipcode'] = summary_df['Zipcode'].astype(int)
final_df = pd.merge(cases_2017_zipcode, summary_df, on="Zipcode")
final_df.to_csv("final_df.csv")
final_df

,CaseID,Opened,Category,Request Type,Neighborhood,Latitude,Longitude,Source,Media URL,Opened Date,Opened Year,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,10395976,01/23/2019 01:34:53 PM,Sign Repair,Sign - Missing,Mission,37.764091,-122.413002,Web,NaN,01/23/2019,2019,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
1,11886797,01/01/2020 09:06:00 PM,General Request - PUBLIC WORKS,request_for_service,Mission,37.764840,-122.415038,Web,NaN,01/01/2020,2020,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
2,11089240,07/05/2019 03:12:37 PM,Graffiti,Graffiti on Building_other,Mission,37.755848,-122.414551,Integrated Agency,NaN,07/05/2019,2019,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
3,11239390,08/07/2019 01:25:51 PM,Graffiti,Graffiti on Building_other,Mission,37.755703,-122.416397,Integrated Agency,NaN,08/07/2019,2019,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
4,11886078,01/01/2020 03:27:00 PM,Street and Sidewalk Cleaning,General Cleaning,Mission,37.769532,-122.415847,Mobile/Open311,http://mobile311.sfgov.org/reports/11886078/ph...,01/01/2020,2020,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873838,11445821,09/21/2019 08:53:04 AM,Encampments,Encampment Reports,Presidio National Park,37.806462,-122.450988,Mobile/Open311,http://mobile311.sfgov.org/reports/11445821/ph...,09/21/2019,2019,94129,4019.0,29.4,190167.0,81974.0,122.0,3.035581
1873839,11451330,09/22/2019 06:16:00 PM,Street Defects,Pavement_Defect,Presidio National Park,37.804123,-122.472125,Mobile/Open311,NaN,09/22/2019,2019,94129,4019.0,29.4,190167.0,81974.0,122.0,3.035581
1873840,11416625,09/14/2019 05:20:44 PM,Parking Enforcement,Blocking_Bicycle_Lane,Presidio National Park,37.801830,-122.456973,Mobile/Open311,http://mobile311.sfgov.org/reports/11416625/ph...,09/14/2019,2019,94129,4019.0,29.4,190167.0,81974.0,122.0,3.035581
1873841,11444928,09/20/2019 07:55:37 PM,Noise Report,Noise Issue,Presidio National Park,37.808779,-122.471963,Mobile/Open311,NaN,09/20/2019,2019,94129,4019.0,29.4,190167.0,81974.0,122.0,3.035581


# We will use the same final_df.csv generated in this notebook for our hypothesis analysis.